In [1]:
import json 

# retrieve password and connection string
with open('/Users/vanes/ThisPC/Documents/GitHub/credentials.json') as f: 
    data = json.load(f)
    mongo_conn_str = data['mongodb']

In [3]:
import pymongo

# connect to mongodb and database
client = pymongo.MongoClient(mongo_conn_str)
db = client.admin

# much of the regex code adopted from many people in the discussion board, including Daniel and Chuan
# I could not get this code to fully run for the life of me
import re 
result = client['Chirps']['Chirps'].aggregate([
    {
        'addFields': {
            'sentiment': {
                '$switch': {
                    'branches': [
                        {
                            'case': {
                                '$regexMatch': {
                                    'input': '$comment', 
                                    'regex': (r"I really love")
                                }
                            },
                            'then': 1
                        }, { 
                            'case': {
                                '$regexMatch': { 
                                    'input': '$comment', 
                                    'regex': (r"I love")
                                }
                            },
                            'then': 2 
                        }, { 
                            'case': { 
                                '$regexMatch' : { 
                                    'input' : '$comment', 
                                    'regex' : (r"I like")
                                }
                            }, 
                            'then': 3
                        }, { 
                            'case': {
                                '$regexMatch' : {
                                    'input': '$comment', 
                                    'regex': (r"I think I might like")
                                }
                            }, 
                            'then': 3 
                        }, { 
                            'case': { 
                                '$regexMatch': { 
                                    'input': '$comment', 
                                    'regex': (r"maybe I like") 
                                } 
                            }, 
                            'then': 4
                        }, { 
                            'case': { 
                                '$regexMatch': { 
                                    'input': '$comment', 
                                    'regex': (r"I don't like") 
                                }
                            }, 
                            'then': 5
                        }, {
                            'case': {
                                '$regexMatch': {
                                    'input': '$comment',
                                    'regex': (r"I hate") 
                                }
                            }, 
                            'then': 6
                        }, { 
                            'case': {
                                'regexMatch': { 
                                    'input': '$comment',
                                    'regex': (r"I really hate")
                                } 
                            }, 
                            'then': 7
                        }
                    ], 
                    'default': 0
                } 
            } 
        } 
    }, { 
        '$match': { 
            'comment': re.compile(r"hiking"), 
            'likes': { 
                '$gt': 10 
            } 
        } 
    }, { 
        '$addFields': { 
            'year': { 
                '$substr': [ 
                    '$date', 0, 4 
                ] 
            }, 
            'month': { 
                '$substr': [ 
                    '$date', 5, 2 
                ] 
            }, 
            'day': { 
                '$substr': [ 
                    '$date', 8, 2
                ] 
            }
        } 
    }, { 
        '$group': { 
            '_id': { 
                'year': '$year', 
                'month': '$month'
            }, 
            'total_chirps': { 
            '$sum': 1 
            }, 
            'average_sentiment': { 
            '$avg': '$sentiment' 
            }
        }
    }
]) 

for i in result: 
    print(i) 
        

NameError: name 'mongo_conn_str' is not defined